In [53]:
import pandas as pd
import numpy as np

In [36]:
df = pd.read_csv('bank-full.csv')

In [37]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [38]:
df = df.drop(columns=["default", "loan"])

In [39]:
df

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


In [40]:
education_mode = df['education'].mode()[0]
education_mode

'secondary'

In [50]:
df1 = df['age'].corr(df['balance'])
df2 = df['day'].corr(df['campaign'])
df3 = df['day'].corr(df['pdays'])
df4 = df['pdays'].corr(df['previous'])
max(df1, df2, df3, df4)

np.float64(0.45481963548050097)

In [56]:
df['y'] = df['y'].replace({'yes': 1, 'no': 0}).astype(int)

In [59]:
from sklearn.model_selection import train_test_split

# Предположим, что ваш DataFrame называется df
# Отделяем целевую переменную y и оставляем только признаки
X = df.drop(columns=['y'])  # Features
y = df['y']  # Target variable

# 1. Split the data into training (60%) and temporary (40%: 20% validation + 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# 2. Split the temporary data into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Проверка размеров полученных наборов данных
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 27126
Validation set size: 9042
Test set size: 9043


In [64]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Предположим, что ваш DataFrame называется df и у вас уже есть разделенные выборки
# Используем только обучающую выборку
X_train = X_train.copy()  # Обучающие признаки
y_train = y_train.copy()  # Обучающая целевая переменная

# Создаем экземпляр LabelEncoder
le = LabelEncoder()

# Преобразуем категориальные переменные в числовые
X_train['contact'] = le.fit_transform(X_train['contact'])
X_train['education'] = le.fit_transform(X_train['education'])
X_train['housing'] = le.fit_transform(X_train['housing'])
X_train['poutcome'] = le.fit_transform(X_train['poutcome'])

# Вычисляем взаимную информацию для каждой переменной отдельно
mi_contact = round(mutual_info_classif(X_train[['contact']], y_train, random_state=42)[0], 2)
mi_education = round(mutual_info_classif(X_train[['education']], y_train, random_state=42)[0], 2)
mi_housing = round(mutual_info_classif(X_train[['housing']], y_train, random_state=42)[0], 2)
mi_poutcome = round(mutual_info_classif(X_train[['poutcome']], y_train, random_state=42)[0], 2)

# Печатаем результаты
print(f"Mutual Information Score for 'contact': {mi_contact}")
print(f"Mutual Information Score for 'education': {mi_education}")
print(f"Mutual Information Score for 'housing': {mi_housing}")
print(f"Mutual Information Score for 'poutcome': {mi_poutcome}")

# Находим переменную с наибольшим значением взаимной информации
max_score = max(mi_contact, mi_education, mi_housing, mi_poutcome)
if max_score == mi_contact:
    max_var = 'contact'
elif max_score == mi_education:
    max_var = 'education'
elif max_score == mi_housing:
    max_var = 'housing'
else:
    max_var = 'poutcome'

print(f"The variable with the biggest mutual information score is: {max_var} with a score of {max_score}")


Mutual Information Score for 'contact': 0.02
Mutual Information Score for 'education': 0.01
Mutual Information Score for 'housing': 0.01
Mutual Information Score for 'poutcome': 0.03
The variable with the biggest mutual information score is: poutcome with a score of 0.03
